# Breast Cancer Classification

## Load Dataset

In [1]:
from sklearn.datasets import load_breast_cancer
import numpy as np
X_raw, y_raw = load_breast_cancer(return_X_y=True)
print(X_raw.shape, y_raw.shape)

(569, 30) (569,)


### Regulate Dataset

In [2]:
X = X_raw / X_raw.max(axis=0)
y = y_raw.reshape(-1, 1)
print(X.shape, y.shape)

(569, 30) (569, 1)


## Make a Model

In [23]:
# Define linear function
def linear(input, weight, bias):
    return input @ weight.T + bias

# Define ReLU function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Redefine forward pass. Intermediate result, Z, needs to be tracked 
def forward(feature, weight, bias):
    Z = linear(feature, weight, bias)
    pred = sigmoid(Z)
    return pred

# Sanity check
w_dummy = np.random.normal(0, 1e-3, (1, X.shape[1]))
b_dummy = np.random.normal(0, 1e-3)
pred_dummy = forward(X, w_dummy, b_dummy)
pred_class_dummy = pred_dummy > 0.5
accuracy = np.sum(pred_class_dummy==y) / y.shape[0]
print(accuracy)

0.8471001757469244


## Evaluate Model with Binary Cross-Entropy Loss

In [24]:
def binary_cross_entropy(pred, label):
    error = -label * np.log(pred) - (1 - label) * np.log(1 - pred)
    return np.mean(error)

# Sanity check
loss_dummy = binary_cross_entropy(pred_dummy, y)
print(loss_dummy)

0.6929241962559888
